# Setup

Install all needed dependencies.

In [1]:
!pip install --quiet --upgrade transformers datasets faiss-cpu


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


In [1]:
# Install the YAML magic
!pip install --quiet yamlmagic
%load_ext yamlmagic


[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: pip install --upgrade pip


Configuration and used models.

In [2]:
%%yaml parameters

# Model
context_encoder_model_name_or_path: facebook/dpr-ctx_encoder-multiset-base
generator_model_name_or_path: facebook/rag-sequence-nq

<IPython.core.display.Javascript object>

#  Transformers RAG

Prepare chunk dataset.

In [3]:
import urllib.request
from datasets import Dataset

link = "https://huggingface.co/ngxson/demo_simple_rag_py/raw/main/cat-facts.txt"
dataset_list = []

# Retrieve knowledge from provided link, use every line as a separate chunk.
for line in urllib.request.urlopen(link):
    dataset_list.append({"text": line.decode('utf-8'), "title": "cats"})

print(f'Loaded {len(dataset_list)} entries')

dataset = Dataset.from_list(dataset_list)

Loaded 150 entries


Encode dataset chunks into embeddings (vector representations), append embeddings into dataset.

Add faiss index for similarity search.

In [4]:
from transformers import (
    DPRContextEncoder,
    DPRContextEncoderTokenizerFast,
)
import torch


torch.set_grad_enabled(False)

ctx_encoder = DPRContextEncoder.from_pretrained(parameters['context_encoder_model_name_or_path'])
ctx_tokenizer = DPRContextEncoderTokenizerFast.from_pretrained(parameters['context_encoder_model_name_or_path'])
ds_with_embeddings = dataset.map(lambda example: {'embeddings': ctx_encoder(**ctx_tokenizer(example["text"], return_tensors="pt"))[0][0].numpy()})
ds_with_embeddings.add_faiss_index(column='embeddings')


None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


ModuleNotFoundError: No module named 'torch'

**Specify user query here**

In [103]:
input_query = "what is the name of the tiniest cat"

Generate response for user query using context from dataset.

In [104]:
from transformers import RagTokenizer, RagRetriever, RagSequenceForGeneration

tokenizer = RagTokenizer.from_pretrained(parameters['generator_model_name_or_path'])

# Construct retriever to return relevant context from dataset
retriever = RagRetriever.from_pretrained(
    parameters['generator_model_name_or_path'], index_name="custom", indexed_dataset=ds_with_embeddings
)

model = RagSequenceForGeneration.from_pretrained(parameters['generator_model_name_or_path'], retriever=retriever)

input_dict = tokenizer.prepare_seq2seq_batch(input_query, return_tensors="pt")

generated = model.generate(input_ids=input_dict["input_ids"])
print(tokenizer.batch_decode(generated, skip_special_tokens=True)[0])

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'RagTokenizer'. 
The class this function is called from is 'BartTokenizer'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called fr

 mr. pebbles
